<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting" data-toc-modified-id="Setting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting</a></span></li></ul></div>

In [1]:
import sys
import os
import time
import datetime
import pickle
import json
import re
import operator
from collections import defaultdict

import pprint as pp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


sys.path.append("/Users/xuzhu/Desktop/code/assistants") # my package
from toolbox.os_assistant import scan_folder

---

## Setting

In [2]:
DATA_FOLDER = "/Users/xuzhu/Desktop/data/open_dataset"